In [2]:
from helper import DataPreprocessing
from ipywidgets import interact, IntSlider
from librosa.feature import melspectrogram
import librosa
import matplotlib.pyplot as plt
import numpy as np
#%matplotlib nbagg

In [3]:
def peak_frequency (spec,freq):
    peak_freqs = np.zeros(spec.shape[1])
    for i in range(spec.shape[1]):
        idx = np.argmax(spec[:, i])
        peak_freqs[i] = freq[idx] 
    return peak_freqs

In [4]:
@interact(kind = ["train","full","test"], n_fft = IntSlider(min=100,max=25600,step=1,value=2560),hop = IntSlider(min=10,max=2560,step=1,value=256), w_name = ["hann","boxcar","triang"], n_mels = IntSlider(min=10,max=500,step = 1, value= 64))
def choose_kind_data(kind, n_fft,hop,w_name,n_mels):

    #load data
    data = DataPreprocessing(kind)
    
    #load method
    mt = "Mel Spectrogram"
    method = "MS"
    
    #load parameter:
    w_size = data.window_size_stft
    sr = data.sr

    if kind in ["train", "full"]:
        if kind == "train":
            bearing_number =  [11,12,21,22,31,32]
        else:
            bearing_number =  [13,14,15,16,17,23,24,25,26,27,33]
        @interact(bearing = bearing_number)
        def load_data_train(bearing):
            
            #load data
            N = data.n_samples(kind,bearing)
            eol = data.eol(bearing)
            time_stamp = np.linspace(0,eol,N)
            norm = data.stand(bearing)
            hor_acc = norm[0]
            ver_acc = norm[1]

            @interact(label = IntSlider(min=0,max=eol-90,step=10))
            def visualize_MS (label):

                #scale data with label and step
                s_step = N - label*sr//100 - w_size 
                time_stamp_w = time_stamp[s_step:s_step+w_size]
                time_stamp_w_real = np.linspace((eol-label)-90,(eol-label)+0.1,w_size)
                hor_acc_w = hor_acc[s_step:s_step+w_size]
                ver_acc_w = ver_acc[s_step:s_step+w_size]
                t_step = time_stamp_w_real.min()

                #visualize
                fig, ((ax1, ax2, ax3, ax4),(ax5, ax6, ax7, ax8)) = plt.subplots(nrows = 2, ncols = 4,figsize = (25,10))
                fig.suptitle("{} data, bearing {}, EOL {}s, {} samples\nMethod {}".format(kind,bearing,eol,N,mt))

                #axes 1:
                ax1.plot(time_stamp,hor_acc)
                ax1.plot(time_stamp_w,hor_acc_w)
                ax1.legend(["horizontal","window"],loc="upper left")
                ax1.set_ylabel("Normalized value")
                ax1.set_xlabel("Timestamp")
                ax1.set_title("Horizontal Acceleromete, time step {}".format(int(t_step)))

                #axes 2:
                ax2.plot(time_stamp_w_real,hor_acc_w,"darkorange")
                ax2.legend(["horizontal window"],loc = "upper left")
                ax2.set_ylabel("Normalized value")
                ax2.set_xlabel("Timestamp window")
                ax2.set_title("Horizontal Acceleromete Windowm, label {}".format(label))
                ax2.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax2.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

                #axes 3:
                ms_hor = melspectrogram(y=hor_acc_w,sr=sr,n_fft=n_fft,hop_length = hop,window=w_name,n_mels=n_mels,htk = True)
                ms_db = librosa.power_to_db(ms_hor)
                mel_f = librosa.mel_frequencies(n_mels=n_mels,fmin=0,fmax=sr/2)
                #image_ms_hor = librosa.display.specshow(data=ms_db,sr=sr,hop_length=hop,n_fft=n_fft,x_axis="time",y_axis="mel",cmap = None, ax= ax3)
                image_ms_hor = ax3.pcolormesh(np.linspace((eol-label)-90,(eol-label)+0.1,ms_db.shape[1]),mel_f,ms_db)
                cbar = plt.colorbar(image_ms_hor,ax=ax3)
                cbar.set_label('dB')
                ax3.set_title("{}, size {}, label {}".format(method,ms_hor.shape,0))
                ax3.set_xlabel("Time domain")
                
                #axes 4:
                peak_hor = peak_frequency(ms_db,mel_f)
                ax4.plot(np.linspace((eol-label),(eol-label)+0.1,ms_db.shape[1]),peak_hor)
                ax4.set_xlabel("Time Domain")
                ax4.set_ylabel("Peak Frequency")
                ax4.set_title("Peak Frequency")

                #axes 5:
                ax5.plot(time_stamp,ver_acc)
                ax5.plot(time_stamp_w,ver_acc_w)
                ax5.legend(["vertikal","window"],loc="upper left")
                ax5.set_ylabel("Normalized value")
                ax5.set_xlabel("Timestamp")
                ax5.set_title("Vertikal Acceleromete,time step {}".format(int(t_step)))
                
                #axes 6:
                ax6.plot(time_stamp_w_real,ver_acc_w,"darkorange")
                ax6.legend(["vertikal window"],loc = "upper left")
                ax6.set_ylabel("Normalized value")
                ax6.set_xlabel("Timestamp window")
                ax6.set_title("Horizontal Acceleromete Window label {}".format(label))
                ax6.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax6.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

                #axes 7:
                ms_ver = melspectrogram(y=ver_acc_w,sr=sr,n_fft=n_fft,hop_length = hop,window=w_name,n_mels=n_mels, htk = True)
                ms_db = librosa.power_to_db(ms_ver)
                #image_ms_ver = librosa.display.specshow(data=ms_db,sr=sr,hop_length=hop,n_fft=n_fft,x_axis="time",y_axis="mel",cmap = None, ax= ax7)
                image_ms_ver = ax7.pcolormesh(np.linspace((eol-label)-100,(eol-label)+0.1,ms_db.shape[1]),mel_f,ms_db)
                cbar=plt.colorbar(image_ms_ver,ax=ax7)
                cbar.set_label('dB')
                ax7.set_title("{}, size {}".format(method,ms_ver.shape))

                #axes 8:
                peak_hor = peak_frequency(ms_db,mel_f)
                ax8.plot(np.linspace((eol-label),(eol-label)+0.1,ms_db.shape[1]),peak_hor)
                ax8.set_xlabel("Time Domain")
                ax8.set_ylabel("Peak Frequency")
                ax8.set_title("Peak Frequency")

                print("Done") 

    elif kind == "test":
        @interact(bearing = [13,14,15,16,17,23,24,25,26,27,33])
        def load_data_test(bearing):
            
            #load data
            N = data.n_samples(kind,bearing)
            last_cycle = data.last_cycle(bearing)
            eol = data.eol(bearing)
            rul = data.rul(bearing)
            time_stamp = np.linspace(0,last_cycle,N)
            norm = data.stand(bearing)
            w_size = data.window_size *10
            hor_acc = norm[0]
            ver_acc = norm[1]
            sr = data.sr

            @interact(label = IntSlider(min=rul,max=rul+last_cycle,step=10))
            def visualize_STFT (label):

                #scale data with label and step
                s_step =  N - (label-rul)*sr//100 - w_size 
                time_stamp_w = time_stamp[s_step:s_step+w_size]
                time_stamp_w_real = np.linspace((last_cycle-label+rul)-90,(last_cycle-label+rul)+0.1,w_size)
                hor_acc_w = hor_acc[s_step:s_step+w_size]
                ver_acc_w = ver_acc[s_step:s_step+w_size]
                t_step = time_stamp_w_real.min()

                #visualize
                fig, ((ax1, ax2, ax3, ax4),(ax5, ax6, ax7, ax8)) = plt.subplots(nrows = 2, ncols = 4,figsize = (25,10))
                fig.suptitle("{} data, bearing {}, EOL {} s, RUL {} s, {} samples\nMethod {}".format(kind,bearing,eol,rul,N,mt))

                #axes 1:
                ax1.plot(time_stamp,hor_acc)
                ax1.plot(time_stamp_w,hor_acc_w)
                ax1.legend(["horizontal","window"],loc="upper left")
                ax1.set_ylabel("Normalized value")
                ax1.set_xlabel("Timestamp")
                ax1.set_title("Horizontal Acceleromete, step {}".format(int(t_step)))

                #axes 2:
                ax2.plot(time_stamp_w_real,hor_acc_w,c="darkorange")
                ax2.legend(["horizontal window"],loc = "upper left")
                ax2.set_ylabel("Normalized value")
                ax2.set_xlabel("Timestamp window")
                ax2.set_title("Horizontal Acceleromete Window, label {}".format(label))
                ax2.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax2.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

                #axes 3:
                ms_hor = melspectrogram(y=hor_acc_w,sr=sr,n_fft=n_fft,hop_length = hop,window=w_name,n_mels=n_mels,hkt = True)
                ms_db = librosa.power_to_db(ms_hor)
                mel_f = librosa.mel_frequencies(n_mels=n_mels,fmin=0,fmax=sr/2)
                #image_ms_hor = librosa.display.specshow(data=ms_db,sr=sr,hop_length=hop,n_fft=n_fft,x_axis="time",y_axis="mel",cmap = None, ax= ax3)
                image_ms_hor = ax3.pcolormesh(np.linspace((last_cycle-label+rul)-100,(last_cycle-label+rul)+0.1,ms_db.shape[1]),mel_f,ms_db)
                cbar = plt.colorbar(image_ms_hor,ax=ax3)
                cbar.set_label('dB')
                ax3.set_title("{}, size {}".format(method,ms_hor.shape))

                #axes 4:
                peak_hor = peak_frequency(ms_db,mel_f)
                ax4.plot(np.linspace((eol-label),(eol-label)+0.1,ms_db.shape[1]),peak_hor)
                ax4.set_xlabel("Time Domain")
                ax4.set_ylabel("Peak Frequency")
                ax4.set_title("Peak Frequency")

                #axes 5:
                ax5.plot(time_stamp,ver_acc)
                ax5.plot(time_stamp_w,ver_acc_w)
                ax5.legend(["vertikal","window"],loc="upper left")
                ax5.set_ylabel("Normalized value")
                ax5.set_xlabel("Timestamp")
                ax5.set_title("Vertikal Acceleromete, step {}".format(int(t_step)))
                
                #axes 6:
                ax6.plot(time_stamp_w_real,ver_acc_w,c="darkorange")
                ax6.legend(["vertikal window"],loc = "upper left")
                ax6.set_ylabel("Normalized value")
                ax6.set_xlabel("Timestamp")
                ax6.set_title("Horizontal Acceleromete Window, label {}".format(label))
                ax6.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax6.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])
                
                #axes 7:
                ms_ver = melspectrogram(y=ver_acc_w,sr=sr,n_fft=n_fft,hop_length = hop,window=w_name,n_mels=n_mels, htk = True)
                ms_db = librosa.power_to_db(ms_ver)
                #image_ms_ver = librosa.display.specshow(data=ms_db,sr=sr,hop_length=hop,n_fft=n_fft,x_axis="time",y_axis="mel",cmap = None, ax= ax7)
                image_ms_ver = ax7.pcolormesh(np.linspace((last_cycle-label+rul)-100,(last_cycle-label+rul)+0.1,ms_db.shape[1]),mel_f,ms_db)
                cbar=plt.colorbar(image_ms_ver,ax=ax7)
                cbar.set_label('dB')
                ax7.set_title("{}, size {}".format(method,ms_ver.shape))

                #axes 8:
                peak_hor = peak_frequency(ms_db,mel_f)
                ax8.plot(np.linspace((eol-label),(eol-label)+0.1,ms_db.shape[1]),peak_hor)
                ax8.set_xlabel("Time Domain")
                ax8.set_ylabel("Peak Frequency")
                ax8.set_title("Peak Frequency")

                print("Done")

interactive(children=(Dropdown(description='kind', options=('train', 'full', 'test'), value='train'), IntSlide…